In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

# Define a custom loss function
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
    
    def forward(self, outputs, targets):
        # This custom loss adds a regularization term to MSE
        mse_loss = torch.mean((outputs - targets) ** 2)
        reg_loss = torch.mean(torch.abs(outputs))  # L1 regularization
        return mse_loss + 0.01 * reg_loss

# Generate some dummy data
input_size = 10
hidden_size = 20
output_size = 1
num_samples = 1000

X = torch.randn(num_samples, input_size)
y = torch.sum(X, dim=1, keepdim=True) + torch.randn(num_samples, 1) * 0.1

# Create model, loss function, and optimizer
model = SimpleNet(input_size, hidden_size, output_size)
criterion = CustomLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    test_input = torch.randn(1, input_size)
    prediction = model(test_input)
    print(f'Test input sum: {test_input.sum().item():.4f}')
    print(f'Model prediction: {prediction.item():.4f}')